In [ ]:
from dotenv import load_dotenv
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai.tools import tool
from IPython.display import display, Markdown
import requests

load_dotenv()

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
synth_llm_small = LLM(
    model="anthropic/claude-3-5-haiku-20241022",
    temperature=0.2,
)
synth_llm_large = LLM(
    model="anthropic/claude-3-5-haiku-20241022",
    temperature=0.7
)

In [4]:
finhealth_sales_analyst_agent = Agent(
    config=agents_config['finhealth_sales_analyst'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=synth_llm_small,
    max_rpm=5,
    respect_context_window=True
)
chief_sales_analyst_agent = Agent(
    config=agents_config['chief_sales_analyst'],
    llm=synth_llm_large,
    max_rpm=25,
    respect_context_window=True
)

In [5]:
research_tasks = [
    Task(config=tasks_config[tc], agent=finhealth_sales_analyst_agent, async_execution=True) \
    for tc in list(tasks_config.keys()) \
    if tc.startswith('finhealth_research')
]

finhealth_report_task = Task(
   config=tasks_config['finhealth_report_task'],
    agent=chief_sales_analyst_agent,
    context=research_tasks
)

In [6]:
finhealth_crew = Crew(
    agents=[finhealth_sales_analyst_agent],
    tasks=[
        *research_tasks,
        finhealth_report_task
    ],
    verbose=True,
    max_rpm=25,
    full_output=True
)

In [7]:
company = 'Virginia Credit Union'

In [ ]:

result = finhealth_crew.kickoff(inputs={
  'Company': company
})

In [9]:
with open('output/%s.md'%company, 'w') as file:
#    file.write(str(result))
    for _task in research_tasks:
        file.write("\n\n\n-------------------------------------------------\n\n\n")     
        file.write(_task.output.raw)